***Phakeme Sithole - ST10090464 - PDAN8412 - POE PART 3***

In [1]:
# Phakeme Sithole - ST10090464 - PDAN8412 - POE PART 3
# IMPORTS
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import os
import zipfile
import sys
import shutil
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from google.colab import files

# 1. DATA UPLOAD & EXTRACTION
print("--- DATA UPLOAD ---")
print("Please upload the 'archive.zip' file for the Flowers dataset.")
uploaded = files.upload()

if len(uploaded) == 0:
    print("No file uploaded. Stopping execution.")
    sys.exit()

zip_filename = list(uploaded.keys())[0]
print(f"Uploaded: {zip_filename}")

print("Unzipping dataset...")
extract_path = 'dataset_extracted'
if os.path.exists(extract_path):
    shutil.rmtree(extract_path)
os.makedirs(extract_path)

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("Unzipping complete.")

# 2. SMART DIRECTORY SEARCH
# I wrote this function to find the images no matter how the zip is structured.
def find_image_root(base_path):
    for root, dirs, files in os.walk(base_path):
        # I check if this folder contains sub-folders (potential classes)
        if len(dirs) > 1:
            # I check if the first sub-folder actually contains images
            first_sub = os.path.join(root, dirs[0])
            if os.path.exists(first_sub) and len(os.listdir(first_sub)) > 0:
                # I check if the files inside are actually images
                sample_file = os.listdir(first_sub)[0]
                if sample_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    return root
    return None

print("Searching for image folders...")
data_dir = find_image_root(extract_path)

if data_dir is None:
    print("Error: Could not find the flower folders automatically.")
    print("Please ensure you uploaded the correct 'Flowers Recognition' zip file.")
    sys.exit()

print(f"Found dataset at: {data_dir}")
class_names = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])
print(f"Classes found: {class_names}")

# 3. EXPLORATORY DATA ANALYSIS (EDA)
img_height, img_width = 150, 150
batch_size = 32

# I verify the balance of the dataset.
class_counts = {}
for c in class_names:
    class_path = os.path.join(data_dir, c)
    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    class_counts[c] = len(images)

plt.figure(figsize=(10, 6))
sns.barplot(x=list(class_counts.keys()), y=list(class_counts.values()))
plt.title("Distribution of Images per Flower Category")
plt.xlabel("Flower Type")
plt.ylabel("Number of Images")
plt.show()

# I visualise random samples to check quality.
plt.figure(figsize=(12, 8))
for i, class_name in enumerate(class_names):
    class_path = os.path.join(data_dir, class_name)
    img_files = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    if img_files:
        img_path = os.path.join(class_path, img_files[0])
        img = load_img(img_path, target_size=(img_height, img_width))

        plt.subplot(2, 3, i + 1)
        plt.imshow(img)
        plt.title(class_name)
        plt.axis("off")
plt.suptitle("Sample Images from Each Class")
plt.show()

# 4. DATA SPLITTING & PREPROCESSING
# Since this dataset is in one big folder, I split it into Train and Validation manually.
print("Preparing data generators...")

# I apply data augmentation to the training set to make the model robust.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2, # I use 20% of data for testing
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

# Training Generator (80% of data)
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse',
    subset='training',
    shuffle=True
)

# Validation Generator (20% of data)
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse',
    subset='validation',
    shuffle=False
)

# 5. CNN MODEL CREATION
model = Sequential()
model.add(Input(shape=(img_height, img_width, 3)))

# Convolutional Block 1
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

# Convolutional Block 2
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

# Convolutional Block 3
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

# Flattening
model.add(Flatten())

# Dense Layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5)) # Prevents overfitting

# Output Layer (Dynamic size based on number of classes found)
model.add(Dense(len(class_names), activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# 6. MODEL TRAINING

print("Starting training... (This may take a few minutes)")
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator
)

# I plot the accuracy and loss.
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.legend()
plt.show()

# 7. EVALUATION

print("Evaluating model on validation set...")
val_loss, val_acc = model.evaluate(validation_generator, verbose=2)
print(f"Final Validation Accuracy: {val_acc*100:.2f}%")

# I generate the classification report.
y_true = validation_generator.classes
predictions = model.predict(validation_generator)
y_pred = np.argmax(predictions, axis=1)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

# I plot the Confusion Matrix.
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# 8. USER INFERENCE SYSTEM
print("--- USER INFERENCE SYSTEM ---")
print("Please upload a photo of a flower to test the model.")
uploaded_inference = files.upload()

if len(uploaded_inference) > 0:
    for fn in uploaded_inference.keys():
        path = fn
        img = load_img(path, target_size=(img_height, img_width))

        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0

        prediction = model.predict(img_array)
        predicted_class_index = np.argmax(prediction)
        predicted_label = class_names[predicted_class_index]
        confidence = np.max(prediction) * 100

        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Prediction: {predicted_label} ({confidence:.2f}%)")
        plt.show()
        print(f"Identified as: {predicted_label}")
else:
    print("No file uploaded.")

    #•	Keras., 2023. The Sequential Model. Documentation Available at: https://keras.io/guides/sequential_model/ [Accessed 30 October 2025].
    #•	TensorFlow., 2025. Image Classification. Tutorial Available at: https://www.tensorflow.org/tutorials/images/classification [Accessed 30 October 2025].
    #•	TensorFlow., 2025. tf.keras.preprocessing.image.ImageDataGenerator. Documentation Available at: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator [Accessed 30 October 2025].
    #•	Keras., 2025. Adam Optimizer. Documentation Available at: https://keras.io/api/optimizers/adam/ [Accessed 30 October 2025].
    #•	Matplotlib., 2025. Pyplot tutorial. Documentation Available at: https://matplotlib.org/stable/tutorials/introductory/pyplot.html [Accessed 30 October 2025].


KeyboardInterrupt: 